In [1]:
#URLから予測したレースのidを記入する
race_id_text = '202307020611'

In [2]:
#レース日を記入する
date_text = "2023年3月26日" #〇年〇月〇日

In [3]:
#予測したいレースのURLを記入
URL = "https://race.netkeiba.com/race/shutuba.html?race_id=202307020611&rf=race_list"#race_listのページを指定

予測したいレースをスクレイピングするコード

In [4]:
#インポート文
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import re

import os
import warnings
import time
import requests

from google.cloud import storage

##### 予測したいレースの設定

In [5]:
#google cloudの設定
key_name ="../key/storage_key.json"
bucket_name = "keiba_csv_upload_bucket"
#raceの保存場所
race_file_upload = 'target_race_data.csv'
race_file_name = '../data/main/target_race_data.csv'
#horseの保存場所
horse_file_upload = 'target_horse_data.csv'
horse_file_name = '../data/main/target_horse_data.csv'

##### スクレイピングの設定

In [6]:
#ドライバーの設定
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,3)

C:\Users\kawam\AppData\Local\Temp\ipykernel_14448\738581181.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
#ページの立ち上げ
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [8]:
#レースのラウンド数を取得
race_round = driver.find_elements(By.CLASS_NAME,'RaceNum')
race_round_text = [rr.text for rr in race_round]
race_round_text = race_round_text[0]

In [9]:
#レースのタイトルを取得
race_title = driver.find_elements(By.CLASS_NAME,'RaceName')
race_title_text = [rr.text for rr in race_title]
race_title_text = race_title_text[0]

In [10]:
#レースの詳細情報一覧を取得
race_detail1 = driver.find_elements(By.CLASS_NAME,'RaceData01')
race_details1_text = [rr.text for rr in race_detail1]

In [11]:
#詳細情報をリストに分ける
race_details1_list = race_details1_text[0].split(' / ')

In [12]:
#発走時間情報の取得
time_text = race_details1_list[0].strip("発走")
time_text = '発走 : '+time_text

In [13]:
#レースコース情報を取得
race_details1_list1_list = race_details1_list[1].split(' ')

In [14]:
#レースコース情報から距離を抜き出す
race_details1_list1_list_text = str(race_details1_list1_list[0])
text = race_details1_list1_list_text
match = re.search(r'[^\d]*(\D)', text)
if match:
    ground_type_text = match.group(1)
else:
    ground_type_text = ''

text = race_details1_list1_list_text
match = re.search(r'\d+', text)
if match:
    distance = match.group()
else:
    distance = ''
distance_int = int(distance)

In [15]:
#レースコース情報から障害レースかどうかを判断
text = race_details1_list1_list_text
is_obstacle = 1 if text.startswith('障') else 0
is_obstacle_int = int(is_obstacle)

In [16]:
#レースコース情報から左右どちら周りかを判断
is_left_right_straight_text = race_details1_list1_list[1].strip("()")

In [17]:
#天気の情報を取得
weather_text = race_details1_list[2]

In [18]:
#地面の状態の情報を取得
ground_status_text = race_details1_list[3]

In [19]:
#レース情報を取得
race_details2 = driver.find_elements(By.CLASS_NAME,'RaceData02')
race_details2_text = [rr.text for rr in race_details2]
race_details2_text = re.sub(' +', ' ', race_details2_text[0])
race_details2_list = race_details2_text.split(' ')

In [20]:
#レース情報も文字列に変換
race_details2_list8_str = str(race_details2_list[7])

In [21]:
#レース情報をグルーピング
race_details2_list8_str
match = re.match(r'^\d+', race_details2_list8_str)
if match:
    race_details2_list[7] = match.group()

In [22]:
#何頭立てかの情報を取得
total_horse_number_text = race_details2_list[7]

In [23]:
#何レース目かなどの情報を取得
where_racecourse_text = race_details2_list[0]+race_details2_list[1]+race_details2_list[2]

In [24]:
#辞書のリストを作成
race_data = []
race_data.append({
        'race_id':race_id_text,
        'race_round': race_round_text,
        'race_title':race_title_text,
         'weather':weather_text,
         'ground_status':ground_status_text,
        'time':time_text,
        'where_racecourse':where_racecourse_text,
        'total_horse_number':total_horse_number_text,
        'ground_type':ground_type_text,
        'is_left_right_straight':is_left_right_straight_text,
        'distance':distance_int,
        'is_obstacle':is_obstacle_int,     
    })

# 辞書のリストをDataFrameに変換する
race_df = pd.DataFrame(race_data)

In [25]:
#race_dfに指定した日付をくっつける
race_df['date'] = date_text

In [26]:
race_df

,race_id,race_round,race_title,weather,ground_status,time,where_racecourse,total_horse_number,ground_type,is_left_right_straight,distance,is_obstacle,date
0,202307020611,11R,高松宮記念,天候:小雨,馬場:不,発走 : 15:40,2回中京6日目,18,芝,左,1200,0,2023年3月26日


##### data-cleaning

In [27]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   race_id                 1 non-null      object
 1   race_round              1 non-null      object
 2   race_title              1 non-null      object
 3   weather                 1 non-null      object
 4   ground_status           1 non-null      object
 5   time                    1 non-null      object
 6   where_racecourse        1 non-null      object
 7   total_horse_number      1 non-null      object
 8   ground_type             1 non-null      object
 9   is_left_right_straight  1 non-null      object
 10  distance                1 non-null      int64 
 11  is_obstacle             1 non-null      int64 
 12  date                    1 non-null      object
dtypes: int64(2), object(11)
memory usage: 232.0+ bytes


##### race_id

##### race_round 

In [28]:
#Rの文字を削除
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [29]:
#objectをintに変換
race_df['race_round'] = race_df['race_round'].astype(int)

##### race_title

##### weather

In [30]:
#天候の文字を削除
race_df['weather'] = race_df['weather'].str.strip('天候 :')

In [31]:
#雨と雪かを判断するカラムを作成
weather_rain = race_df["weather"].str.extract('(小雨|雨)', expand=True)
weather_snow = race_df["weather"].str.extract('(小雪|雪)', expand=True)
weather_rain.columns ={"weather_rain"}
weather_snow.columns ={"weather_snow"}
race_df = pd.concat([race_df, weather_rain], axis=1)
race_df = pd.concat([race_df, weather_snow], axis=1)

In [32]:
#上記のnullを0で埋める
race_df.fillna(value={'weather_rain': 0}, inplace=True)
#小雨は1
race_df['weather_rain'] = race_df['weather_rain'].replace('小雨', 1)
#雨は2に置換
race_df['weather_rain'] = race_df['weather_rain'].replace('雨', 2)

In [33]:
#上記のnullを0で埋める
race_df.fillna(value={'weather_snow': 0}, inplace=True)
#小雪は1
race_df['weather_snow'] = race_df['weather_snow'].replace('小雪', 1)
#雪は2に置換
race_df['weather_snow'] = race_df['weather_snow'].replace('雪', 2)

In [34]:
#weatherを削除
race_df.drop(['weather'], axis=1, inplace=True)

##### ground_status

In [35]:
#それぞれのステータスを数字に置換
race_df['ground_status'] = race_df['ground_status'].replace('.*(稍重).*', 4,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(重).*', 3,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(不).*', 2,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(不良).*', 2,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(良).*', 1,regex=True)

In [36]:
race_df['ground_status']

0    2
Name: ground_status, dtype: int64

##### time

In [37]:
#timeとdateの情報からdatetimeを作成
warnings.filterwarnings('ignore', 'The default value of regex')
race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')
race_df["datetime"] = race_df["date"] + race_df["time"]
race_df["datetime"] = pd.to_datetime(race_df['datetime'], format='%Y年%m月%d日%H時%M分')

In [38]:
#timeとdateを削除
race_df.drop(['time'], axis=1, inplace=True)
race_df.drop(['date'], axis=1, inplace=True)

##### where_racecourse

In [39]:
#会場の情報だけ抜き出す
race_df["where_racecourse"] = race_df["where_racecourse"].str.replace('\d*回(..)\d*日目', r'\1')

In [40]:
#それぞれを数字に変換
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*札幌.*', 1,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*函館.*', 2,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*福島.*', 3,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*新潟.*', 4,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*東京.*', 5,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*中山.*', 6,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*中京.*', 7,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*京都.*', 8,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*阪神.*', 9,regex=True)
race_df['where_racecourse'] = race_df['where_racecourse'].replace('.*小倉.*', 10,regex=True)
#1札幌　2函館　3福島　4新潟　5東京　6中山　7中京　8京都　9阪神　10小倉　

#####  total_horse_number 

In [41]:
#objectをintに変換
race_df['total_horse_number'] = race_df['total_horse_number'].astype(int)

##### ground_type

In [42]:
#それぞれを数字に変換
race_df['ground_type'] = race_df['ground_type'].replace('.*芝.*', 0,regex=True)
race_df['ground_type'] = race_df['ground_type'].replace('.*ダ.*', 1,regex=True)
race_df['ground_type'] = race_df['ground_type'].replace('.*障.*', 2,regex=True)
race_df['ground_type'] 

0    0
Name: ground_type, dtype: int64

##### is_left_right_straight

In [43]:
#上記のnullを0で埋める
race_df.fillna(value={'is_left_right_straight': 0}, inplace=True)
#それぞれを数字に変換
race_df['is_left_right_straight'] = race_df['is_left_right_straight'].replace('.*右.*', 1,regex=True)
race_df['is_left_right_straight'] = race_df['is_left_right_straight'].replace('.*左.*', 2,regex=True)

In [44]:
race_df

,race_id,race_round,race_title,ground_status,where_racecourse,total_horse_number,ground_type,is_left_right_straight,distance,is_obstacle,weather_rain,weather_snow,datetime
0,202307020611,11,高松宮記念,2,7,18,0,2,1200,0,1,0,2023-03-26 15:40:00


In [45]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   race_id                 1 non-null      object        
 1   race_round              1 non-null      int32         
 2   race_title              1 non-null      object        
 3   ground_status           1 non-null      int64         
 4   where_racecourse        1 non-null      int64         
 5   total_horse_number      1 non-null      int32         
 6   ground_type             1 non-null      int64         
 7   is_left_right_straight  1 non-null      int64         
 8   distance                1 non-null      int64         
 9   is_obstacle             1 non-null      int64         
 10  weather_rain            1 non-null      int64         
 11  weather_snow            1 non-null      int64         
 12  datetime                1 non-null      datetime64[ns]

In [46]:
#csvの書き出し
race_df.to_csv("../data/main/target_race_data.csv", index=False)

##### google cloud

In [47]:
# サービスアカウントキーを指定
client = storage.Client.from_service_account_json(key_name)

In [48]:
#その他の情報を指定
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(race_file_upload)
blob.upload_from_filename(race_file_name)

# horse

In [49]:
#BeautifulSoupの設定
load_url = URL
html = requests.get(load_url)
soup = BeautifulSoup(html.content, "html.parser")

In [50]:
#保存リストを作成
horse_list = []
result_row  = []
horse_list_waku = []
horse_list_num = []
horse_list_horse_id = []
horse_list_agesex = []
horse_list_weight = []
horse_list_rider_id = []
horse_list_stable_id = []
horse_list_horseweight = []
horse_list_odds = []
horse_list_popluar = []
horse_list_horse_name = []

In [51]:
#tableの情報を取得
horse_tables = soup.findAll("table", class_="RaceTable01")
horse_table = horse_tables[0].findAll('tr', class_="HorseList")

In [52]:
#tableの情報から各情報を抜き出す
for i in range(len(horse_table)):
    result_row = horse_table[i].findAll("td")
    horse_list_waku.append(result_row[0].get_text())#枠
    horse_list_num.append(result_row[1].get_text())#番号
    horse_list_horse_id.append(result_row[3].find('a').get('href').split("/")[-1])#馬名
    horse_list_agesex.append(result_row[4].get_text())#性別年齢
    horse_list_weight.append(result_row[5].get_text())#斤量
    horse_list_rider_id.append(result_row[6].find('a').get('href').split("/")[-2])#騎手
    horse_list_stable_id.append(result_row[7].find('a').get('href').split("/")[-2])#厩舎
    horse_list_horseweight.append(result_row[8].get_text())#馬体重
    horse_list_odds.append(result_row[9].get_text())#オッズ
    horse_list_popluar.append(result_row[10].get_text())#人気
    horse_list_horse_name.append(result_row[3].get_text())#馬の名前

In [53]:
#辞書のリストを作成
horse_data = []
for i in range(len(horse_table)):
    horse_data.append({
            'frame_number':horse_list_waku[i],
            'horse_number':horse_list_num[i],
            'horse_id':horse_list_horse_id[i],
            'sex_and_age':horse_list_agesex[i],
            'burden_weight':horse_list_weight[i],
            'rider_id':horse_list_rider_id[i],
            'tamer_id':horse_list_stable_id[i],
            'horse_weight':horse_list_horseweight[i],
            'odds':horse_list_odds[i],
            'popular':horse_list_popluar[i],
            'horse_name':horse_list_horse_name[i]
        })
# 辞書のリストをDataFrameに変換する
horse_df = pd.DataFrame(horse_data)

In [54]:
#race_idを入力
horse_df['race_id'] = race_id_text

In [55]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   frame_number   18 non-null     object
 1   horse_number   18 non-null     object
 2   horse_id       18 non-null     object
 3   sex_and_age    18 non-null     object
 4   burden_weight  18 non-null     object
 5   rider_id       18 non-null     object
 6   tamer_id       18 non-null     object
 7   horse_weight   18 non-null     object
 8   odds           18 non-null     object
 9   popular        18 non-null     object
 10  horse_name     18 non-null     object
 11  race_id        18 non-null     object
dtypes: object(12)
memory usage: 1.8+ KB


##### Cleaning

##### frame_number

In [56]:
#intに変換
horse_df['frame_number'] = horse_df['frame_number'].astype(int)

##### horse_number

In [57]:
#intに変換
horse_df['horse_number'] = horse_df['horse_number'].astype(int)

##### horse_id

In [58]:
#horse_df['horse_id'] = horse_df['horse_id'].astype(int)

##### sex_and_age

In [59]:
#数字のみ抜き出し
horse_df["age"] = horse_df["sex_and_age"].str.extract('([0-9]+)', expand=True)
horse_df["age"] = horse_df["age"] .astype(int)

In [60]:
#それぞれのステータスを数字に置換
horse_df["sex_and_age"]  = horse_df["sex_and_age"].replace('.*(牡).*', 0,regex=True)
horse_df["sex_and_age"]  = horse_df["sex_and_age"].replace('.*(牝).*', 1,regex=True)
horse_df["sex_and_age"]  = horse_df["sex_and_age"].replace('.*(セ).*', 2,regex=True)

In [61]:
#sexを作成
horse_df["sex"] = horse_df["sex_and_age"]

In [62]:
#sex_and_ageを削除
horse_df.drop(["sex_and_age"] , axis=1, inplace=True)

##### burden_weight

In [63]:
horse_df['burden_weight'] = horse_df['burden_weight'].astype(float)

In [64]:
horse_df['burden_weight'] = horse_df['burden_weight'].astype(int)

##### rider_id

In [65]:
#horse_df['rider_id'] = horse_df['rider_id'].astype(int)

##### tamer_id

In [66]:
#horse_df['tamer_id'] = horse_df['tamer_id'].astype(int)

##### horse_weight

In [67]:
#()ないの文字を抜き出して、差分を取得
horse_weight_dif = horse_df["horse_weight"].str.extract('\(([-|+]?\d*)\)', expand=True)
horse_weight_dif.columns ={"horse_weight_dif"}
horse_df = pd.concat([horse_df, horse_weight_dif], axis=1)
#horse_weight_difを作成
horse_df['horse_weight_dif'] = horse_df['horse_weight_dif'].astype(int)

In [68]:
#不要な文字の削除
horse_df['horse_weight'] = horse_df['horse_weight'].replace('\(([-|+]?\d*)\)', '', regex=True)
horse_df['horse_weight'] = horse_df['horse_weight'].replace('計不', np.nan)
horse_df['horse_weight'] = horse_df['horse_weight'].astype(int)

##### odds

##### popular

In [69]:
#oddsとpopularの情報を削除 ->ほかのファイルにまとめる
horse_df = horse_df.drop(['odds','popular'],axis=1)

##### horse_name

In [70]:
horse_df['horse_name'] = horse_df['horse_name'].str.strip()

In [71]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   frame_number      18 non-null     int32 
 1   horse_number      18 non-null     int32 
 2   horse_id          18 non-null     object
 3   burden_weight     18 non-null     int32 
 4   rider_id          18 non-null     object
 5   tamer_id          18 non-null     object
 6   horse_weight      18 non-null     int32 
 7   horse_name        18 non-null     object
 8   race_id           18 non-null     object
 9   age               18 non-null     int32 
 10  sex               18 non-null     int64 
 11  horse_weight_dif  18 non-null     int32 
dtypes: int32(6), int64(1), object(5)
memory usage: 1.4+ KB


In [72]:
horse_df

,frame_number,horse_number,horse_id,burden_weight,rider_id,tamer_id,horse_weight,horse_name,race_id,age,sex,horse_weight_dif
0,1,1,2015104364,58,01091,01135,484,トゥラヴェスーラ,202307020611,8,0,-12
1,1,2,2019102788,56,01095,01160,470,ウォーターナビレラ,202307020611,4,1,-16
2,2,3,2015101028,58,01018,01027,502,キルロード,202307020611,8,2,-6
3,2,4,2018105610,58,01019,01120,512,ダディーズビビッド,202307020611,5,0,-2
4,3,5,2018105069,56,01032,01161,486,メイケイエール,202307020611,5,1,9
5,3,6,2016106453,58,01117,00420,492,ナランフレグ,202307020611,7,0,-6
6,4,7,2017100434,58,01171,01113,520,ヴェントヴォーチェ,202307020611,6,0,6
7,4,8,2017110144,56,05203,01183,484,ロータスランド,202307020611,6,1,6
8,5,9,2017100583,58,01191,00427,478,ディヴィナシオン,202307020611,6,0,6
9,5,10,2017100381,56,01169,01064,476,オパールシャルム,202307020611,6,1,-10


In [73]:
#csvに書き出し
horse_df.to_csv("../data/main/target_horse_data.csv", index=False)

In [74]:
#保存場所の指定
blob = bucket.blob(horse_file_upload)
blob.upload_from_filename(horse_file_name)